In [9]:
!pip install z3-solver

# Codificação lógica de um programa
Considere o seguinte programa C sobre inteiros.

```
z = 0;
x = x + y;
if (y >= 0) {
  y = x - y;
  x = x - y;
}
else {
  z = x - y;
  x = y;
  y = 0;
}
z = x + y + z;
```
Faça a codificação lógica deste programa.


Conversão para SA:

```
z1 = 0;
x1 = x0  y0;
if (y0 >= 0) {
  y1 = x1 - y0;
  x2 = x1 - y1;
}
else {
  z2 = x1 - y0;
  x3 = y0;
  y2 = 0;
}
x4 = (y0 >= 0) ? x2 : x3;
y3 = (y0 >= 0) ? y1 : y2;
z3 = (y0 >= 0) ? z1 : z2;
z4 = x4 + y3 + z3;

```

Conversão para CNF:

```
z1 = 0;
x1 = x0 + y0;
if (y0 >= 0) y1 = x1 - y0;
if (y0 >= 0) x2 = x1 - y1;

if !(y0 >= 0) z2 = x1 - y0;
if !(y0 >= 0) x3 = y0;
if !(y0 >= 0) y2 = 0;

x4 = (y0 >= 0) ? x2 : x3;
y3 = (y0 >= 0) ? y1 : y2;
z3 = (y0 >= 0) ? z1 : z2;
z4 = x4 + y3 + z3;

```

Codificação lógica do programa em Z3:

In [10]:
from z3 import *


x0, x1, x2, x3, x4 = Ints('x0 x1 x2 x3 x4')
y0, y1, y2, y3 = Ints('y0 y1 y2 y3')
z1, z2, z3, z4 = Ints('z1 z2 z3 z4')

s = Solver()

s.add(z1 == 0)
s.add(x1 == x0 + y0)

s.add(Implies(y0 >= 0, y1 == x1 - y0))
s.add(Implies(y0 >= 0, x2 == x1 - y1))

s.add(Implies(Not(y0 >= 0), z2 == x1 - y0))
s.add(Implies(Not(y0 >= 0), x3 == y0))
s.add(Implies(Not(y0 >= 0), y2 == 0))

s.add(x4 == If(y0 >= 0,x2,x3))
s.add(y3 == If(y0 >= 0,y1,y2))
s.add(z3 == If(y0 >= 0,z1,z2))

s.add(z4 == x4 + y3 + z3)

if s.check() == sat :
    print("Ok.")
else:
    print("A codificação não deve estar bem.")

Ok.


Tendo por base a codificação lógica que fez do programa, utilize o API do Z3 para Python para se pronunciar quanto à veracidade das seguintes afirmações. Justifique a sua resposta. No caso da afirmação ser falsa, apresente o contra-exemplo indicado pelo solver.

   1. *Se o valor inicial de $y$ for positivo, o programa faz a troca dos valores de $x$ e $y$ entre si.*

   2. *O valor final de $y$ nunca é negativo.*
   
   3. *O valor final de $z$ corresponde à soma dos valores de entrada de $x$ e $y$.*

   4. *O valor final de x é sempre negativo.*

In [11]:
s.push()

prop = "\n\"Se o valor inicial de y for positivo, o programa faz a troca \
dos valores de x e y entre si.\""

form = Implies(y0>0, And(x4==y0,y3==x0))
s.add(Not(form))

if s.check() == sat :
    print("A afirmação: ", prop)
    print("é falsa.")
    m = s.model()
    print("Por exemplo:")
    for d in m.decls():
        print("%s = %d" % (d.name(), m[d].as_long()))
else:
    print("A afirmação: ", prop)
    print("é verdadeira.")


s.pop()

A afirmação:  
"Se o valor inicial de y for positivo, o programa faz a troca dos valores de x e y entre si."
é verdadeira.


In [12]:
s.push()

prop = "\n\"O valor final de y nunca é negativo.\""

form = y3 >= 0
s.add(Not(form))

if s.check() == sat :
    print("A afirmação: ", prop)
    print("é falsa.")
    m = s.model()
    print("Por exemplo, com o seguinte valor inicial para as variáveis:")
    print("x=%d, y=%d" % (m[x0].as_long(),m[y0].as_long()))
    print("O valor final de y será %d." % m[y3].as_long())
else:
    print("A afirmação: ", prop)
    print("é verdadeira.")

s.pop()

A afirmação:  
"O valor final de y nunca é negativo."
é falsa.
Por exemplo, com o seguinte valor inicial para as variáveis:
x=-1, y=1
O valor final de y será -1.


In [13]:
s.push()

prop = "\n\"O valor final de z corresponde à soma dos valores de entrada de x e y.\""

form = z4 == x0 + y0
s.add(Not(form))

if s.check() == sat :
    print("A afirmação: ", prop)
    print("é falsa.")
    m = s.model()
    print("Por exemplo")
    for d in m.decls():
        print("%s= %d" % (d.name(), m(d).as_long()))
else:
    print("A afirmação: ", prop)
    print("é verdadeira.")

s.pop()

A afirmação:  
"O valor final de z corresponde à soma dos valores de entrada de x e y."
é verdadeira.


In [14]:
s.push()

prop = "\n\"O valor final de x é sempre negativo.\""


form = x4 < 0
s.add(Not(form))

if s.check() == sat :
    print("A afirmação: ", prop)
    print("é falsa.")
    m = s.model()
    print("Por exemplo, com o seguinte valor inicial para as variáveis:")
    print("x=%d, y=%d" % (m[x0].as_long(),m[y0].as_long()))
    print("O valor final de x será %d." % m[x4].as_long())
else:
    print("A afirmação: ", prop)
    print("é verdadeira.")

s.pop()

A afirmação:  
"O valor final de x é sempre negativo."
é falsa.
Por exemplo, com o seguinte valor inicial para as variáveis:
x=-1, y=0
O valor final de x será 0.
